In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import requests
from pathlib import Path

def TPG_Trigger(scatter_x_value,
                scatter_y_value,
                scatter_x_label,
                scatter_y_label,
                ratio_x_value,
                ratio_y_value,
                ratio_x_label,
                ratio_y_label,
                comparison_x_value,
                comparison_y_value,
                comparison_x_label,
                comparison_y_label,
                dates,
                multiple_run_predictions_length,
                plot_name):
    headers={'Accept-Encoding': 'gzip, deflate, br','Content-Type': 'application/json','Accept': 'application/json','Connection': 'keep-alive','DNT': '1'}
    query = """
        query PlotMe(
            $scatter_x_value: [Float]!,
            $scatter_y_value: [Float]!,
            $scatter_x_label: String!,
            $scatter_y_label: String!,
            $ratio_x_value: [Float]!,
            $ratio_y_value: [Float]!,
            $ratio_x_label: String!,
            $ratio_y_label: String!,
            $comparison_x_value: [Float]!,
            $comparison_y_value: [Float]!,
            $comparison_x_label: String!,
            $comparison_y_label: String!,
            $dates: [String]!,
            $multiple_run_predictions_length: Int!,
            $plot_name: String!
        ) {
            plotme(
                scatter_x_value: $scatter_x_value,
                scatter_y_value: $scatter_y_value,
                scatter_x_label: $scatter_x_label,
                scatter_y_label: $scatter_y_label,
                ratio_x_value: $ratio_x_value,
                ratio_y_value: $ratio_y_value,
                ratio_x_label: $ratio_x_label,
                ratio_y_label: $ratio_y_label,
                comparison_x_value: $comparison_x_value,
                comparison_y_value: $comparison_y_value,
                comparison_x_label: $comparison_x_label,
                comparison_y_label: $comparison_y_label,
                dates: $dates,
                multiple_run_predictions_length: $multiple_run_predictions_length,
                plot_name: $plot_name
            ) {
                success
                errors
                    plots {
      plot_num
      plot_image
      plot_trend_slope
      plot_trend_intercept
      plot_trend_correlation
      plot_trend_standard_deviation
      plot_trend_dispersion
    }
            }
        }
    """

    variables = {
        "scatter_x_value": scatter_x_value,
        "scatter_y_value": scatter_y_value,
        "scatter_x_label": scatter_x_label,
        "scatter_y_label": scatter_y_label,
        "ratio_x_value": ratio_x_value,
        "ratio_y_value": ratio_y_value,
        "ratio_x_label": ratio_x_label,
        "ratio_y_label": ratio_y_label,
        "comparison_x_value": comparison_x_value,
        "comparison_y_value": comparison_y_value,
        "comparison_x_label": comparison_x_label,
        "comparison_y_label": comparison_y_label,
        "dates": dates,
        "multiple_run_predictions_length": multiple_run_predictions_length,
        "plot_name" : plot_name
    }
    data = {
        'query': query,
        'variables' : variables
    }

    response = requests.post('http://3.64.122.73:5002/graphql',headers=headers,json=data)
    print(response.text)
    return response.json()


def preprocess_data(dataset,look_back=60):
    from sklearn.preprocessing import MinMaxScaler
    dataset = dataset.values
    dataset = dataset.astype('float32')
    #normalize
    scaler = MinMaxScaler(feature_range=(0,1))
    dataset = scaler.fit_transform(dataset)
    #create windows 
    X,y = [],[]
    for i in range(len(dataset) - look_back):
        features = dataset[i:(i+look_back),0]
        target = dataset[i+look_back,0]
        X.append(features)
        y.append(target)

    return scaler,np.array(X), np.array(y)

In [2]:
data_dir = Path('../data')

df = pd.read_csv(
    data_dir / 'SPP_MSFT.csv',
    usecols=['DNCP','MPN5P']
)

df['DATE'] = pd.to_datetime(df['DNCP'], origin='1899-12-30', unit='D')
df['DATE'] = df['DATE'].astype(str)
df = df.drop('DNCP',axis=1)
df = df.set_index('DATE')

df.head()

,MPN5P
DATE,
2000-02-29,46.765625
2000-03-01,47.070312
2000-03-02,47.656250
2000-03-03,48.000000
2000-03-06,48.750000


In [3]:
from keras.models import load_model

model = load_model('../models/raw_features_lstmv1.h5')

scaler,X,y = preprocess_data(df,look_back=60)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
preds = model.predict(X)

preds = scaler.inverse_transform(preds)
actual = scaler.inverse_transform(y.reshape(-1,1))

y_actual = preds.flatten().tolist()
y_pred = preds.flatten().tolist()

182/182 [==============================] - 5s 20ms/step


In [4]:

response = TPG_Trigger(
    scatter_x_value=y_actual,
    scatter_y_value=y_pred,
    scatter_x_label='X',
    scatter_y_label='Y',
    ratio_x_value=y_actual,
    ratio_y_value=y_pred,
    ratio_x_label='X',
    ratio_y_label='Y',
    comparison_x_value=y_actual,
    comparison_y_value=y_pred,
    comparison_x_label='X',
    comparison_y_label='Y',
    dates=df.index.tolist(),
    multiple_run_predictions_length = len(y_pred),
    plot_name='LSTM_Target_Plots'
)

print(response)

{
  "data": {
    "plotme": {
      "errors": null,
      "plots": [
        {
          "plot_image": "...",
          "plot_num": 1,
          "plot_trend_correlation": 1.0,
          "plot_trend_dispersion": 1.0,
          "plot_trend_intercept": 1.0,
          "plot_trend_slope": 1.0,
          "plot_trend_standard_deviation": 60.6283
        },
        {
          "plot_image": "...",
          "plot_num": 2,
          "plot_trend_correlation": 1.0,
          "plot_trend_dispersion": 1.0,
          "plot_trend_intercept": 1.0,
          "plot_trend_slope": 1.0,
          "plot_trend_standard_deviation": 60.6283
        },
        {
          "plot_image": "...",
          "plot_num": 3,
          "plot_trend_correlation": 1.0,
          "plot_trend_dispersion": 1.0,
          "plot_trend_intercept": 1.0,
          "plot_trend_slope": 1.0,
          "plot_trend_standard_deviation": 60.6283
        }
      ],
      "success": true
    }
  }
}

{'data': {'plotme': {'errors': None, 'p